# Fish-Speech-1.5 日本語テキスト→音声変換アプリ（シンプル版）

このノートブックはウィジェットを使用せず、シンプルなインターフェースでFish-Speech-1.5を使用します。

## 準備
以下のセルを実行して、必要なライブラリをインストールします。初回のみ実行してください。

In [ ]:
# 必要なライブラリのインストール（最初に一度だけ実行）
!pip install torch torchaudio numpy scipy requests soundfile

## ライブラリのインポート

In [ ]:
import requests
import numpy as np
import torch
import torchaudio
import os
import io
import tempfile
import time
import json
from IPython.display import Audio, display, HTML
import scipy.io.wavfile as wavfile
import soundfile as sf

## 変換のためのユーティリティ関数

In [ ]:
# Fish Speech APIを呼び出す関数
def text_to_speech_api(text, language="ja", speaker=None):
    """
    Fish Speech Spaces APIを使用して、テキストを音声に変換する
    
    Parameters:
    -----------
    text : str
        変換するテキスト
    language : str
        言語コード（'ja', 'en', 'zh'など）
    speaker : str or None
        話者ID（Noneの場合はデフォルト）
        
    Returns:
    --------
    str
        生成された音声ファイルのパス
    """
    # APIエンドポイント
    API_URL = "https://fishaudio-fish-speech-1.hf.space/api/predict"
    
    # 一時ファイルを作成
    temp_dir = tempfile.gettempdir()
    output_path = os.path.join(temp_dir, 'output_api.wav')
    
    try:
        # APIリクエストのデータを準備
        if not speaker:
            # 言語に基づいてスピーカーを選択
            if language == "ja":
                speaker = "ja_speaker_1"
            elif language == "en":
                speaker = "en_speaker_1"
            elif language == "zh":
                speaker = "zh_speaker_1"
            else:
                speaker = "default"
        
        payload = {
            "data": [
                text,  # テキスト
                "auto",  # 言語自動検出（または直接言語コードを指定）
                speaker,  # 話者ID
                0.6,  # 温度
                1.2,  # トップK
                0.7,  # トップP
                False  # 声色推論（不要ならFalse）
            ]
        }
        
        print(f"APIリクエスト送信中...")
        
        # POSTリクエストを送信
        response = requests.post(API_URL, json=payload)
        
        # レスポンスを確認
        if response.status_code == 200:
            response_data = response.json()
            
            # 音声データのURLを取得
            audio_url = response_data.get('data', [])[1]
            
            if audio_url and audio_url.startswith('http'):
                # 音声データをダウンロード
                audio_response = requests.get(audio_url)
                
                if audio_response.status_code == 200:
                    # 音声データを一時ファイルに保存
                    with open(output_path, 'wb') as f:
                        f.write(audio_response.content)
                    
                    print(f"音声生成完了: {output_path}")
                    return output_path
                else:
                    raise Exception(f"音声データのダウンロードに失敗しました: {audio_response.status_code}")
            else:
                raise Exception(f"音声URLが見つかりません: {response_data}")
        else:
            raise Exception(f"APIリクエストに失敗しました: {response.status_code} - {response.text}")
    
    except Exception as e:
        print(f"APIエラー: {e}")
        
        # エラーが発生した場合はデモ音声を生成
        print("デモ音声を生成します...")
        
        # デモ音声（正弦波）
        sample_rate = 24000
        duration = min(len(text) * 0.1, 10)  # テキストの長さに比例（最大10秒）
        t = np.linspace(0, duration, int(duration * sample_rate), endpoint=False)
        
        if language == "ja":
            freq = 440  # 日本語の場合
        else:
            freq = 392  # その他の言語
            
        waveform = 0.5 * np.sin(2 * np.pi * freq * t) * np.exp(-t/duration)
        wavfile.write(output_path, sample_rate, waveform.astype(np.float32))
        
        print(f"デモ音声を生成しました: {output_path}")
        return output_path

# 速度調整関数
def adjust_audio_speed(input_file, output_file, speed=1.0):
    """
    音声ファイルの再生速度を調整する
    
    Parameters:
    -----------
    input_file : str
        入力音声ファイルのパス
    output_file : str
        出力音声ファイルのパス
    speed : float
        速度調整率（1.0が標準）
        
    Returns:
    --------
    str
        出力音声ファイルのパス
    """
    try:
        # 音声ファイルの読み込み
        data, samplerate = sf.read(input_file)
        
        if speed != 1.0:
            # 速度調整のためのリサンプリング
            # リサンプリングの比率を計算
            resample_ratio = 1.0 / speed
            
            # PyTorchのリサンプラーを使用
            audio_tensor = torch.tensor(data).unsqueeze(0)
            
            if len(audio_tensor.shape) == 3:  # (1, channels, samples)
                audio_tensor = audio_tensor.squeeze(0)
            
            # モノラルかステレオかを確認
            if len(audio_tensor.shape) == 1:  # モノラル
                audio_tensor = audio_tensor.unsqueeze(0)
            
            # リサンプラーの作成
            resampler = torchaudio.transforms.Resample(
                orig_freq=samplerate,
                new_freq=int(samplerate * resample_ratio)
            )
            
            # リサンプリング
            resampled_audio = resampler(audio_tensor)
            
            # NumPy配列に変換
            if resampled_audio.shape[0] == 1:  # モノラル
                output_data = resampled_audio.squeeze(0).numpy()
            else:  # ステレオ
                output_data = resampled_audio.numpy().T
        else:
            # 速度調整なし
            output_data = data
        
        # 音声ファイルの保存
        sf.write(output_file, output_data, samplerate)
        
        return output_file
    except Exception as e:
        print(f"音声速度調整エラー: {e}")
        return input_file

## 1. ファイルからテキストを読み込む方法

In [ ]:
# テキストファイルのパスを指定（ファイルのパスを変更してください）
file_path = "sample_text.txt"  # サンプルテキストまたは任意のテキストファイルへのパス

# ファイルから内容を読み込む
try:
    encodings = ['utf-8', 'shift-jis', 'euc-jp', 'iso-2022-jp']
    text = None
    
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                text = f.read()
                print(f"ファイルをエンコーディング {encoding} で読み込みました")
                break
        except UnicodeDecodeError:
            continue
    
    if text is None:
        raise ValueError("ファイルのエンコーディングを検出できませんでした。")
    
    # テキストを表示
    print("\n===== テキスト内容 =====\n")
    print(text[:1000] + ("..." if len(text) > 1000 else ""))
    
    # 言語の選択（'ja'=日本語, 'en'=英語, 'zh'=中国語)
    language = "ja"  # 必要に応じて変更
    
    # 速度の設定 (1.0 = 通常速度、0.5 = 半分の速度、2.0 = 2倍の速度)
    speed = 1.0
    
    # 音声に変換
    audio_path = text_to_speech_api(text, language=language)
    
    # 速度調整が必要な場合
    if speed != 1.0:
        temp_dir = tempfile.gettempdir()
        speed_adjusted_path = os.path.join(temp_dir, f'output_speed_{speed}.wav')
        audio_path = adjust_audio_speed(audio_path, speed_adjusted_path, speed)
    
    # 音声を表示
    display(Audio(audio_path))
    
    print(f"\n音声ファイルの保存場所: {audio_path}")
    
except Exception as e:
    print(f"エラーが発生しました: {e}")

## 2. 直接テキストを入力する方法

以下のセルのテキストを修正して、変換したいテキストを入力してください。

In [ ]:
# ここに変換したいテキストを入力
input_text = """こんにちは。これはFish-Speech-1.5を使った日本語テキスト読み上げのサンプルです。
このモデルは、高品質な日本語音声合成ができます。"""

# 設定
language = "ja"  # 'ja'=日本語, 'en'=英語, 'zh'=中国語
speed = 1.0      # 1.0=通常速度, 0.5=半分の速度, 2.0=2倍の速度

try:
    # テキストを表示
    print("\n===== 入力テキスト =====\n")
    print(input_text)
    
    # 音声に変換
    audio_path = text_to_speech_api(input_text, language=language)
    
    # 速度調整が必要な場合
    if speed != 1.0:
        temp_dir = tempfile.gettempdir()
        speed_adjusted_path = os.path.join(temp_dir, f'output_speed_{speed}.wav')
        audio_path = adjust_audio_speed(audio_path, speed_adjusted_path, speed)
    
    # 音声を表示
    display(Audio(audio_path))
    
    print(f"\n音声ファイルの保存場所: {audio_path}")
    
except Exception as e:
    print(f"エラーが発生しました: {e}")

## 3. 速度調整を試す方法

さまざまな速度で試してみます。

In [ ]:
# サンプルテキスト
sample_text = "これは速度調整のテストです。同じテキストをさまざまな速度で読み上げます。"

# テキストを一度だけ音声に変換
base_audio_path = text_to_speech_api(sample_text, language="ja")

# 異なる速度で表示
speeds = [0.5, 0.8, 1.0, 1.2, 1.5, 2.0]

for speed in speeds:
    try:
        print(f"\n速度: {speed}x")
        
        # 速度調整
        temp_dir = tempfile.gettempdir()
        speed_adjusted_path = os.path.join(temp_dir, f'output_speed_{speed}.wav')
        adjusted_path = adjust_audio_speed(base_audio_path, speed_adjusted_path, speed)
        
        # 音声を表示
        display(Audio(adjusted_path))
        
    except Exception as e:
        print(f"エラーが発生しました: {e}")

## 4. 異なる言語を試す方法

In [ ]:
# 異なる言語のサンプルテキスト
languages = {
    "ja": "こんにちは、これは日本語のサンプルテキストです。",
    "en": "Hello, this is a sample text in English.",
    "zh": "你好，这是一个中文示例文本。"
}

for lang_code, text in languages.items():
    try:
        print(f"\n言語: {lang_code} - {text}")
        
        # 音声に変換
        audio_path = text_to_speech_api(text, language=lang_code)
        
        # 音声を表示
        display(Audio(audio_path))
        
    except Exception as e:
        print(f"エラーが発生しました: {e}")